In [4]:
import numpy as np
import pandas as pd

In [5]:
df_data = pd.read_csv('airline-passengers.csv')
df_data.tail()

,Month,Passengers
139,1960-08,606
140,1960-09,508
141,1960-10,461
142,1960-11,390
143,1960-12,432


In [6]:
df_passenger= df_data['Passengers']

In [7]:
# transform univariate time series to supervised learning problem
from numpy import array
# split a univariate sequence into samples
def split_sequence(sequence, n_steps):
    X, y = list(), list()
    for i in range(len(sequence)):
    # find the end of this pattern
        end_ix = i + n_steps
    # check if we are beyond the sequence
        if end_ix > len(sequence)-1:
            break
    # gather input and output parts of the pattern
        # print(i, end_ix)
        seq_x, seq_y = sequence[i:end_ix], sequence[end_ix]
        X.append(seq_x)
        y.append(seq_y)
    return array(X), array(y)

In [8]:
n_steps = 13
X, y = split_sequence(df_passenger, n_steps)

In [9]:
# column names to X and y data frames
df_X = pd.DataFrame(X, columns=['t-'+str(i) for i in range(n_steps-1, -1,-1)])
df_y = pd.DataFrame(y, columns=['t+1 (prediction)'])

# concat df_X and df_y
df = pd.concat([df_X, df_y], axis=1)

In [10]:
df.head(3)

,t-12,t-11,t-10,t-9,t-8,t-7,t-6,t-5,t-4,t-3,t-2,t-1,t-0,t+1 (prediction)
0,112,118,132,129,121,135,148,148,136,119,104,118,115,126
1,118,132,129,121,135,148,148,136,119,104,118,115,126,141
2,132,129,121,135,148,148,136,119,104,118,115,126,141,135


In [11]:
from sklearn.preprocessing import MinMaxScaler
scaler= MinMaxScaler()
X_norm= scaler.fit_transform(df_X)
# y_norm= scaler.fit_transform(df_y)

In [12]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_norm, y, test_size=0.2, random_state=0)

In [13]:
df_train = pd.DataFrame(X_train, columns=['t-'+str(i) for i in range(n_steps-1, -1,-1)])
df_train['t+1 (prediction)'] = y_train
df_train.tail(3)

,t-12,t-11,t-10,t-9,t-8,t-7,t-6,t-5,t-4,t-3,t-2,t-1,t-0,t+1 (prediction)
101,0.415385,0.340659,0.274725,0.217582,0.274725,0.303297,0.283516,0.358242,0.318533,0.320463,0.407336,0.492126,0.458661,312
102,0.560440,0.452747,0.512088,0.562637,0.523077,0.663736,0.641758,0.694505,0.710425,0.857143,0.878378,0.687008,0.576772,362
103,0.197802,0.202198,0.202198,0.290110,0.287912,0.274725,0.305495,0.351648,0.324324,0.256757,0.206564,0.129921,0.171260,204


In [14]:
df_train.to_csv('dftrain.csv', index=False)

In [15]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
# fit the model
from sklearn.neighbors import KNeighborsRegressor


model_knn = KNeighborsRegressor(n_neighbors=4)


In [16]:
model_knn.fit(X_train, y_train)
y_pred=model_knn.predict(X_test)

In [17]:
y_pred

array([144.75, 244.5 , 444.5 , 360.25, 276.75, 212.  , 212.25, 181.75,
       217.25, 206.  , 395.  , 150.75, 168.25, 522.25, 133.75, 209.  ,
       340.5 , 343.5 , 135.25, 203.5 , 277.5 , 367.  , 173.25, 267.5 ,
       252.25, 266.75, 332.  ])

In [21]:
df_data.shape

(144, 2)

In [23]:
mae = (1/len(y_test))*np.sum(np.abs(y_test-y_pred))

mae

16.166666666666664

In [18]:
# mean absolute percentage error
from sklearn.metrics import mean_absolute_percentage_error
mean_absolute_percentage_error(y_test, y_pred)

0.06564649033196138